## 3.1. 텍스트 분류:BERT

텍스트 분류(Text classification)는 입력 텍스트를 ***미리 정의된 범주나 레이블로 할당하는 과제***를 의미한다. 즉:주어진 텍스트를 분석하여, 해당 텍스트가 어떤 범주에 속하는지를 예측하는 것이다. 비슷한 사례는 스팸 메일 분류, 감정 분석, 뉴스 기사 분류 등이 있다.

**과제 수행 방법**
1. **데이터 전처리**: 텍스트 데이터를 BERT 모델에 입력할 수 있는 형태로 변환한다.
2. **모델 구축**: BERT 모델을 이용해 텍스트 분류 모델을 구축한다.
3. **모델 학습**: 분류 모델을 학습시킨다.
4. **모델 평가**: 학습된 모델을 평가한다.
5. **모델 예측**: 새로운 데이터에 대해 예측을 수행한다.ㅓ
6. **결과 분석**: 모델의 예측 결과를 분석한다.
7. **추가 고려 사항**: 모델의 성능을 높이기 위한 방법을 고려한다.
8. **배포**: 모델을 서비스에 배포한다.
9. **모델 관리**: 모델을 관리하고 유지보수한다.


### 3.1.1. BERT

BERT(Bidirectional Encoder Representations from Transformers)는 2018년에 공개된 모델로, ***양방향 Transformer 인코더***를 사용한다. BERT는 ***사전 훈련된 언어 모델***로, ***텍스트 분류, 질의응답, 개체명 인식 등 다양한 자연어 처리 과제에 사용***할 수 있다. BERT의 아키텍처는 여러 개의 트랜스포머 인코더 계층이 쌓여있는 구조다. 각 인코더 계층은 양방향 멀티 헤드 셀프 어텐션 매커니즘과 잔차 연결(residual connection)이 적용된 피드 포워드 신경망으로 구성되어 있다. ...

```mermaid
---
title: BERT 모델 구조
---
stateDiagram-v2
    state "입력 텍스트" as input
    state "토큰 & 세그먼트 임베딩" as embedding
    state "위치 인코딩" as position
    state N {
        state "Q" as q
        state "K" as k
        state "V" as v
        state "멀티 헤드 어텐션" as attention
        state "잔차 연결 & 계층 정규화" as norm1
        state "피드 포워드 신경망" as ffnn
        state "잔차 연결 & 계층 정규화" as norm2
        [*] --> q
        [*] --> k
        [*] --> v
        q --> attention
        k --> attention
        v --> attention
        attention --> norm1
        norm1 --> ffnn
        ffnn --> norm2
    }
    state "풀러 계층" as pooler

    [*] --> input
    input --> embedding
    embedding --> position
    position --> N
    N --> pooler
```

BERT의 사전 학습 과정에서는 **마스크된 언어 모델(Masked Language Model, MLM)** 과 **다음 문장 예측(Next Sentence Prediction, NSP)** 이라는 두 가지 학습 방법을 사용한다. MLM은 입력 문장의 일부 단어를 마스킹하고, 마스킹된 단어를 예측하는 방식으로 학습한다. NSP는 두 문장이 이어지는 문장인지 아닌지를 예측하는 방식으로 학습한다. 이러한 사전 학습 과정을 통해 BERT는 다양한 자연어 처리 과제에 적용할 수 있는 표현을 학습한다.

사전 학습이 완료된 BERT모델은 다운스트림 작업별로 전이 학습을 통해 성능을 개선할 수 있다. 텍스트 분류에서는 [CLS] 토큰의 출력을 분류 작업에 사용할 수 있다. [CLS] 토큰은 입력 문장의 전체 정보를 요약한 벡터로, 이를 분류 작업에 사용할 수 있다. 또한 워드피스 임베딩을 사용해 단어를 서브워드 단위로 분리함으로써 OOV(Out-Of-Vocabulary) 문제를 해결할 수 있다.

사전 학습된 BERT모델의 마지막 트랜스포머 인코더 계층에서 출력된[CLS]토큰의 벡터는 전체 입력 텍스트를 요약하는 데 사용된다. 이 벡터는 분류 작업에 사용되는 분류기로 입력되어, 입력 텍스트의 범주를 예측한다. 이러한 방식으로 BERT는 텍스트 분류 작업에 사용될 수 있다.

### 3.1.2. BertTokenizer

BERT는 워드피스 토크나이저를 사용한다. 워드피스는 단어를 더 작은 서브워드 단위로 나누는 방식을 취한다. 예를 들어 'loving'이라는 단어는 'love'와 '##ing'이라는 두 개의 서브워드로 나누어진다. 이러한 서브워드 단위로 토크나이징을 수행하면, OOV 문제를 해결할 수 있다.

`google-bert/bert-base-multilingual-uncased`는 구글에서 102개 언어를 이용해 사전 학습한 다중 언어(Multilingual) 모델로, 다양한 언어의 자연어 처리 과제에서 사용할 수 있다. 



In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-uncased')

text = "Transformers Is so COOL!!!"

encoded = tokenizer(text)
print(encoded)

input_ids = encoded['input_ids'] # 입력 텍스트를 정수 인코딩으로 변환한 값을 의미한다.
decoded = tokenizer.decode(input_ids) # 정수 인코딩을 다시 텍스트로 변환한 값을 의미한다.
print(decoded)

loading file vocab.txt from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/tokenizer_config.json
loading file tokenizer.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/tokenizer.json
loading configuration file config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-multilingual-uncased",
  "

{'input_ids': [101, 58263, 10127, 10297, 26462, 106, 106, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] transformers is so cool!!! [SEP]


### 3.1.3. BertModel

BertModel 클래스를 잉해 사전 학습된 BERT모델을 불러온다. 불러온 BERT모델은 사전학습을 통해 임베딩 계층, 인코더 계층, 풀러 계층이 최적화 되어 있다.

임베딩 계층은 입력 텍스트를 벡터 형태로 변환하는 역할을 한다. 이 계층은 단어 임베딩, 위치 임베딩, 토큰 종류 임베딩 계층으로 구성되어 있다.
단어 임베딩 계층은 입력 토큰을 고유한 임베딩 벡터로 변환하고 위치 임베딩 계층은 각 토큰의 위치 정보를 임베딩해 토큰 간의 순서와 위치 정보를 보존한다. 토큰 종류 임베딩 계층은 입력 토큰의 종류를 구분하는 임베딩 벡터를 생성한다.

인코더 계층은 12개의 트랜스포머 인코더 계층으로 구성되어 있다. 각 인코더 계층은 어텐션 계층, 중간 계층, 출력 계층으로 이루어져 있다. 어텐션 계층은 입력 벡터의 어텐션을 계산하고, 중간 계층은 어텐션 계층의 출력을 처리하는 역할을 한다. 출력 계층은 중간 계층의 출력을 다음 계층으로 전달하는 역할을 한다.

플러계층은 인코더 계층의 최종 출력을 받아 [CLS] 토큰의 벡터를 생성한다. 이 벡터는 분류 작업에 사용되는 분류기로 입력된다.

In [11]:
# BERT모델 구조
from transformers import BertModel

model = BertModel.from_pretrained('google-bert/bert-base-multilingual-uncased')

for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for sub_sub_name, sub_sub_module in sub_module.named_children():
            print("│ └",sub_sub_name)
            for sub_sub_sub_name, sub_sub_sub_module in sub_sub_module.named_children():
                print("│ │ └", sub_sub_sub_name)
                

loading configuration file config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 105879
}

loading weights f

embeddings
└ word_embeddings
└ position_embeddings
└ token_type_embeddings
└ LayerNorm
└ dropout
encoder
└ layer
│ └ 0
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 1
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 2
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 3
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 4
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 5
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 6
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 7
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 8
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 9
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 10
│ │ └ attention
│ │ └ intermediate
│ │ └ output
│ └ 11
│ │ └ attention
│ │ └ intermediate
│ │ └ output
pooler
└ dense
└ activation


이 모델로는 텍스트 분류 과제를 수행할 수 없다. 텍스트 분류에 사용되는 **분류 헤드(classification head)**를 추가해야 한다. 분류 헤드는 [CLS] 토큰의 벡터를 입력으로 받아, 입력 텍스트의 범주를 예측하는 역할을 한다. 분류 헤드란 모델의 출력을 기반으로 입력 텍스트를 다양한 클래스 또는 범주로 분류하는 부분을 의미한다. 이 헤드는 일반적으로 신경망의 마지막 계층에 추가되며, 모델의 출력을 클래스에 대한 확률 분포로 변환하고, 각 클래스에 대한 예측을 수행한다.

분류 헤드를 추가함으로써, 사전에 학습된 모델을 텍스트 분류 과제에 쉽게 적용할 수 있다. 추가 학습 없이도 새로운 텍스트 분류 과제에 빠르게 적용할 수 있다. 트랜스포머 라이브러리에서는 `BertForSequenceClassification` 클래스를 제공해 이러한 분류 헤드를 쉽게 추가할 수 있다.

`BertForSequenceClassification`은 `BertModel`과 달리 텍스트 분류에 필요한 분류 헤드를 가지고 있다. 분류 헤드는 BERT의 출력값 크기를 입력 크기로, 분류할 클래스 수를 출력 크기로 하는 선형 계층이다. 

classifier계층을 통과하면 정답 레이블과 클래스 수에 따라 다른 손실 함수를 사용해 손실값을 계산한다. 손실 함수는 분류 문제의 유형에 따라 적절히 선택해야 한다. 먼저, 분류 할 클래스 수가 1개일 때는 회귀 번석을 위한 평균 제곱 오차 손실 함수(Mean Squared Error, MSE)를 사용한다. 분류할 클래스 수가 2개일 때는 이진 분류를 위한 이진 교차 엔트로피 손실 함수(Binary Cross Entropy, BCE)를 사용한다. 분류할 클래스 수가 3개 이상일 때는 다중 클래스 분류를 위한 크로스 엔트로피 손실 함수(Cross Entropy, CE)를 사용한다.

### 3.1.4. 텍스트 분류 모델 학습

네이버 영화 리뷰 감성 분석 데이터세트로 분류 모델을 학습한다.

네이버 영화 리뷰 감성 분석 데이터세트는 15만개의 학습 데이터와 5만개의 테스트 데이터로 이루어져 있다. 각 데이터는 리뷰를 식별하는 id, 리뷰 내용 텍스트인 document, 긍정, 부정을 나타내는 label로 구성되어 있다. 이 데이터세트를 map메서드를 이용해서 전처리 한다. 토큰화된 텍스트의 길이가 모델의 최대 길이를 초과할 경우, 이를 자르기 위해서 truncation을 True 로 설정한다.

모델 학습에 사용되지 않는 id, document는 제거한다. 그리고 BertForSequenceClassification 모델은 학습에 사용할 레이블을 label이 아닌 labels로 설정한다.



** 사용되는 모델**
BERT multilingual base model (uncased)

102개 언어의 가장 큰 Wikipedia를 사용하여 마스크된 언어 모델링(MLM) 목표로 사전 학습된 모델입니다. 이 모델은 이 논문에서 소개되었으며 이 저장소에서 처음 공개되었습니다. 이 모델은 소문자입니다: 영어와 English를 구분하지 않습니다.


모델 설명

BERT는 다국어 데이터의 대규모 코퍼스에서 자체 지도 방식으로 사전 학습된 트랜스포머 모델입니다. 이는 사람이 어떤 방식으로도 레이블을 지정하지 않은 원시 텍스트만으로 사전 학습되었음을 의미합니다(따라서 많은 공개 데이터를 사용할 수 있음). 이러한 텍스트에서 입력과 레이블을 생성하는 자동 프로세스를 사용합니다. 더 정확히 말하면, 두 가지 목표로 사전 학습되었습니다:

마스크된 언어 모델링(MLM): 문장을 가져와 모델이 입력의 15% 단어를 무작위로 마스킹한 다음 전체 마스크된 문장을 모델에 통과시켜 마스크된 단어를 예측해야 합니다. 이는 전통적인 순환 신경망(RNN)이 보통 단어를 하나씩 보는 것과 다르며, 내부적으로 미래 토큰을 마스킹하는 GPT와 같은 자기 회귀 모델과도 다릅니다. 이를 통해 모델은 문장의 양방향 표현을 학습할 수 있습니다. 다음 문장 예측(NSP): 모델은 사전 학습 중에 두 개의 마스크된 문장을 입력으로 연결합니다. 때로는 원래 텍스트에서 서로 인접한 문장에 해당하고, 때로는 그렇지 않습니다. 그런 다음 모델은 두 문장이 서로 이어지는지 여부를 예측해야 합니다. 이 방식으로 모델은 학습 세트의 언어에 대한 내부 표현을 학습하여 다운스트림 작업에 유용한 기능을 추출할 수 있습니다: 예를 들어, 레이블이 지정된 문장 데이터셋이 있는 경우 BERT 모델이 생성한 기능을 입력으로 사용하여 표준 분류기를 훈련할 수 있습니다.

In [12]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification

def preprocess_data(example, tokenizer):
    """전처리 함수

    Parameters
    * example: dataset의 각 원소
    * tokenizer: 토크나이저

    반환값: 토큰화된 입력 텍스트
    """
    return tokenizer(
        example['document'], # document 키에 해당하는 값이 입력 텍스트
        trucation=True, # 입력 텍스트가 긴 경우, max_length로 자르는 옵션
    )

tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-uncased')

dataset = load_dataset('nsmc', trust_remote_code=True)
processed_dataset = dataset.map(
    lambda x: preprocess_data(x, tokenizer),
    batched=True,
).rename_column('label', 'labels')

print(dataset)
print(processed_dataset)
print(dataset['train'][0])
print(processed_dataset['train'][0])

loading file vocab.txt from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/tokenizer_config.json
loading file tokenizer.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/tokenizer.json
loading configuration file config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-multilingual-uncased",
  "

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})
{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'labels': 0, 'input_ids': [101, 1174, 25539, 23236, 29234, 13045, 119, 119, 87550, 97082, 25539, 1176, 25539, 24937, 13045, 16801, 72197, 47024, 1169, 70724, 22585, 13926, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

배치 내에서 가장 긴 데이터의 길이로 패딩하는 것을 설정한다.

텍스트 데이터를 BERT 모델에 입력하기 위해서는 패딩 작업이 필수적이다. 각 텍스트의 길이가 모두 다르기 때문에 배치 처리를 위해서는 데이터 길이를 동일하게 맞춰야한다. DataCollatorWithPadding을 사용하면 패딩 작업을 쉽게 수행할 수 있다.

첫번째 max_length로 모델의 최대 입력 길이로 모든 데이터를 패딩한다. 이 전략에서는 512의 길이로 패딩이 수행된다.
두번째 전략은 longest로 현재 배치에서 가장 긴 텍스트의 길이로 패딩한다. 따라서 배치마다 패딩 길이가 달라질 수 있다. 

In [21]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, DataCollatorWithPadding

def preprocess_data(example, tokenizer):
    """전처리 함수

    Parameters
    * example: dataset의 각 원소
    * tokenizer: 토크나이저

    반환값: 토큰화된 입력 텍스트
    """
    return tokenizer(
        example['document'], # document 키에 해당하는 값이 입력 텍스트
        trucation=True, # 입력 텍스트가 긴 경우, max_length로 자르는 옵션
    )

tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-uncased')

dataset = load_dataset('nsmc', trust_remote_code=True)
processed_dataset = dataset.map(
    lambda x: preprocess_data(x, tokenizer),
    batched=True,
    remove_columns=["id", "document"],
).rename_column('label', 'labels')

max_length_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length",
)

max_length_dataloader = DataLoader(
    processed_dataset['train'],
    collate_fn=max_length_collator,
    batch_size=4,
    shuffle=False
)

max_length_iterator = iter(max_length_dataloader)
max_length_batch = next(max_length_iterator)

print("max_length 패딩 입력 id shape:", max_length_batch['input_ids'].shape)

longest_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)

longest_dataloader = DataLoader(
    processed_dataset['train'],
    collate_fn=longest_collator,
    batch_size=4,
    shuffle=False
)

longest_iterator = iter(longest_dataloader)
longest_batch = next(longest_iterator)

print("longest 패딩 입력 id shape:", longest_batch['input_ids'].shape)

loading file vocab.txt from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/tokenizer_config.json
loading file tokenizer.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/tokenizer.json
loading configuration file config.json from cache at /home/asanobm/.cache/huggingface/hub/models--google-bert--bert-base-multilingual-uncased/snapshots/7cbf9a625e29989f6b9c6c2fa68234c304f7e38f/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-multilingual-uncased",
  "

max_length 패딩 입력 id shape: torch.Size([4, 512])
longest 패딩 입력 id shape: torch.Size([4, 42])


In [6]:
# 트레이너를 이용해서 택스트 분류 모델 학습하기

import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google-bert/bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.to(device)

dataset = load_dataset('nsmc', trust_remote_code=True)
processed_dataset = dataset.map(
    lambda x: tokenizer(x['document']),
    batched=True,
    remove_columns=["id", "document"],
).rename_column('label', 'labels')

longest_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)

training_args = TrainingArguments(
    output_dir="runs/text-classification",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=1,
    eval_steps=200,
    logging_steps=200,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=longest_collator,
    train_dataset=processed_dataset['train'].select(range(10000)),
    eval_dataset=processed_dataset['test'].select(range(100)),
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google-bert/bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 105879
}



/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
200,0.643000
400,0.581500
600,0.519200


/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=625, training_loss=0.57506826171875, metrics={'train_runtime': 92.4655, 'train_samples_per_second': 108.148, 'train_steps_per_second': 6.759, 'total_flos': 512211447022080.0, 'train_loss': 0.57506826171875, 'epoch': 1.0})

학습 배치 크기는 8, 검증 배치크기는 16으로 설정한다. 학습률은 5e-5로 설정한다. 이외에도 AdamW 옵티마이저를 사용하고, 손실 함수는 크로스 엔트로피 손실 함수를 사용한다.
200스텝마다 로그를 출력하고, 평가를 수행한다. 

**텍스트 분류 수행**

입력 텍스트를 모델에 입력하기 위해서는 토크나이저를 이용해 인코딩한다. 이때 return_tensors인자를 pt로 설정하면 인코딩 결과는 pytorch 텐서로 반환된다.

인코딩된 텍스트 정보를 모델에 입력하면, 모델의 분류 헤드를 통과하며 로짓값을 반환한다. 이 로짓값을 소프트맥스 함수를 통과시켜 확률값으로 변환하면, 각 클래스에 대한 예측 확률을 얻을 수 있다.

평가를 수행할 때는 모델을 평가 모드로 설정하고, 그래디언트 계산을 비활성화한다. 이후 테스트 데이터를 이용해 예측을 수행하고, 예측값과 정답 레이블을 비교해 정확도를 계산한다.

In [8]:
# 텍스트 분류 수행

import torch
model.eval()

text = "이 영화 정말 재미있었어요!"
inputs = tokenizer(text, return_tensors='pt').to(device)

with torch.no_grad():
    outputs = model(**inputs.to(device))
    print(outputs.logits)
    print(outputs.logits.argmax())

tensor([[-1.2073,  2.1918]], device='cuda:0')
tensor(1, device='cuda:0')


In [9]:
import evaluate

yhat = trainer.predict(processed_dataset['test'])
predictions = yhat.predictions.argmax(axis=1)
references = yhat.label_ids

metric = evaluate.load("accuracy")
accuracy = metric.compute(predictions=predictions, references=references)

print(accuracy)

metric = evaluate.load("f1")
f1 = metric.compute(predictions=predictions, references=references)
print(f1)

/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'accuracy': 0.7652}
{'f1': 0.7561178279114213}


학습된 모델을 평가하기 위해 테스트 데이터세트와 트레이너 클래스의 predict 메서드로 감정분석을 수행한다. 예측 결과로는 분류 헤드를 통과한 predictions 와 실제 정답인 references가 반환된다.

허깅페이스의 평가 라이브러리를 사용해 정확도와 F1점수로 모델의 성능을 평가한다. 정확도는 전체 예측중 올바른 예측의 비율을 나타낸다. F1점수는 정밀도와 재현률의 조화 평균으로 모델의 전반적인 성능을 반영한다.